# Episode 48. Neural Network Example: Keras and PyTorch Side-By-Side

# **Method 1. Keras Implementation**

## Import Libraries

In [40]:
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.model_selection import train_test_split

## Read Data

In [41]:
# Load Loan Default Data
# Original Data Source: https://www.kaggle.com/kmldas/loan-default-prediction
from google.colab import files
uploaded = files.upload()

Saving Default_Fin.csv to Default_Fin (2).csv


In [42]:
# Store data in DataFrame
df = pd.read_csv("Default_Fin.csv")

## Cleanup Data

In [43]:
# Convert to numpy
x = df[['Employed','Bank Balance','Annual Salary']].values
y = df[['Defaulted?']].values

In [44]:
# split data: train & test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [45]:
# Standardize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

## Build Neural Network

In [46]:
# Build neural network
model = Sequential()
model.add(Dense(20, input_dim=x.shape[1], activation='relu')) # Hidden 1
model.add(Dense(10, activation='relu')) # Hidden 2
model.add(Dense(y.shape[1],activation='sigmoid')) # Output

model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(x_train,y_train,verbose=2,epochs=100)

Epoch 1/100
235/235 - 1s - loss: 0.3130 - 846ms/epoch - 4ms/step
Epoch 2/100
235/235 - 0s - loss: 0.1132 - 306ms/epoch - 1ms/step
Epoch 3/100
235/235 - 0s - loss: 0.0884 - 308ms/epoch - 1ms/step
Epoch 4/100
235/235 - 0s - loss: 0.0826 - 348ms/epoch - 1ms/step
Epoch 5/100
235/235 - 0s - loss: 0.0807 - 296ms/epoch - 1ms/step
Epoch 6/100
235/235 - 0s - loss: 0.0803 - 309ms/epoch - 1ms/step
Epoch 7/100
235/235 - 0s - loss: 0.0799 - 299ms/epoch - 1ms/step
Epoch 8/100
235/235 - 0s - loss: 0.0798 - 300ms/epoch - 1ms/step
Epoch 9/100
235/235 - 0s - loss: 0.0796 - 298ms/epoch - 1ms/step
Epoch 10/100
235/235 - 0s - loss: 0.0794 - 294ms/epoch - 1ms/step
Epoch 11/100
235/235 - 0s - loss: 0.0796 - 303ms/epoch - 1ms/step
Epoch 12/100
235/235 - 0s - loss: 0.0795 - 309ms/epoch - 1ms/step
Epoch 13/100
235/235 - 0s - loss: 0.0791 - 298ms/epoch - 1ms/step
Epoch 14/100
235/235 - 0s - loss: 0.0794 - 302ms/epoch - 1ms/step
Epoch 15/100
235/235 - 0s - loss: 0.0791 - 310ms/epoch - 1ms/step
Epoch 16/100
235/23

## Evaluate Training Performance

In [50]:
from sklearn.metrics import accuracy_score
y_pred = model.predict(x_test).round()
correct = accuracy_score(y_pred,y_test)
print(f"Accuracy: {correct}")
print("# default:",y_pred.sum())

Accuracy: 0.9728
# default: 41.0


# **Method 2. PyTorch Implementation**

## Import Libraries
```
  # Similar to Keras, but load PyTorch instead
  
  import numpy as np
  import pandas as pd
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense
  from sklearn.model_selection import train_test_split

```

In [1]:
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

## Read Data
```
  # Same as Keras
  
  # Load Loan Default Data
  # Original Data Source: https://www.kaggle.com/kmldas/loan-default-prediction
  from google.colab import files
  uploaded = files.upload()

  # Store data in DataFrame
  df = pd.read_csv("Default_Fin.csv")

```

In [2]:
# Load Loan Default Data
# Original Data Source: https://www.kaggle.com/kmldas/loan-default-prediction
from google.colab import files
uploaded = files.upload()

Saving Default_Fin.csv to Default_Fin (4).csv


In [3]:
# Store data in DataFrame
df = pd.read_csv("Default_Fin.csv")

## Cleanup Data
```
  # Same as Keras

  # Convert to numpy
  x = df[['Employed','Bank Balance','Annual Salary']].values
  y = df[['Defaulted?']].values

  # split data: train & test
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

  # Standardize data
  mean = x_train.mean(axis=0)
  std = x_train.std(axis=0)
  x_train = (x_train - mean) / std
  x_test = (x_test - mean) / std

```

In [4]:
# Convert to numpy - Classification
x = df[['Employed','Bank Balance','Annual Salary']].values
y = df[['Defaulted?']].values

In [5]:
# split data: train & test
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)

In [6]:
# Standardize data
mean = x_train.mean(axis=0)
std = x_train.std(axis=0)
x_train = (x_train - mean) / std
x_test = (x_test - mean) / std

## Build Neural Network
```
  # Unlike Keras, PyTorch uses python class
  
  model = Sequential()
  model.add(Dense(20, input_dim=x.shape[1], activation='relu')) # Hidden 1
  model.add(Dense(10, activation='relu')) # Hidden 2
  model.add(Dense(y.shape[1],activation='sigmoid')) # Output

  model.compile(loss='binary_crossentropy', optimizer='adam')
  model.fit(x_train,y_train,verbose=2,epochs=100)

```

In [7]:
class Net(nn.Module):
    def __init__(self, in_count, out_count):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(in_count, 20)
        self.fc2 = nn.Linear(20, 10)
        self.fc3 = nn.Linear(10, out_count)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return self.sigmoid(x)

In [8]:
# convert to PyTorch Tensor
x_train = Variable(torch.Tensor(x_train).float())
x_test = Variable(torch.Tensor(x_test).float())
y_train = Variable(torch.Tensor(y_train).float())
y_test = Variable(torch.Tensor(y_test).float())

In [9]:
# build model
model = Net(x.shape[1],1)
criterion = nn.MSELoss()# cross entropy loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

## Train Network
```
  # PyTorch directly accesses gradient in neural network
  # uses loss.backward() & optimizer.step()

  model.fit(x_train,y_train,verbose=2,epochs=100)

```

In [ ]:
for epoch in range(100):
    optimizer.zero_grad()
    out = model(x_train)
    loss = criterion(out, y_train)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, loss: {loss.item()}")

## Evaluate Performance
```
  # Similar to Keras, but pytorch tensor to numpy array conversion needed

  from sklearn.metrics import accuracy_score
  y_pred = model.predict(x_test).round()
  correct = accuracy_score(y_pred,y_test)
  print(f"Accuracy: {correct}")

```

In [23]:
from sklearn.metrics import accuracy_score

# predict default in the test data
pred = model(x_test)

# convert PyTorch Tensor to numpy array
np_pred = pred.cpu().detach().numpy().round()
np_ytest = y_test.cpu().detach().numpy().round()

# show accuracy
print("Total # Test Data:", np_ytest.shape[0])
print("# Default (True)",np_ytest.sum())
print("# Default (Predicted)",np_pred.sum())

correct = accuracy_score(np_ytest,np_pred)
print(f"Accuracy: {correct}")

Total # Test Data: 2500
# Default (True) 81.0
# Default (Predicted) 26.0
Accuracy: 0.97
